# Facies discrimination using elemental intensities (design the workflow)
Let's adopt the elemental dataset which having the rolling trick to run the gridsearch using server.

In [1]:
import numpy as np 
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')
#mpl.rcParams['xtick.labelsize'] = 8
#mpl.rcParams["image.cmap"] = 'gray'
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

import datetime
date = datetime.datetime.now().strftime('%Y%m%d')

%matplotlib inline

# Launch deployment

In [24]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
    project="aslee@10.110.16.5",
    queue='main',
    cores=40, 
    memory='10 GB',
    walltime="00:10:00",
    log_directory='job_logs'
)

client = Client(cluster)
cluster.scale(100)

In [23]:
client.close()
cluster.close()

In [25]:
client = Client(cluster)
cluster.scale(100)
#cluster.adapt(maximum=100)

In [29]:
client

Client Scheduler: tcp://10.110.16.11:35853 Dashboard: http://10.110.16.11:8787/status,Cluster Workers: 8 Cores: 40 Memory: 10.00 GB


# Run ML
Just for demonstrating and recording the workflow. The detailed grid search will be carried out using cluster.

There are some problems with mycv....

## No dask dataframe or array

In [11]:
from split import my_train_test_split
data_df = pd.read_csv('data/XRF_ML_cr.csv')
X = data_df.iloc[:, 1:-2].values
y = data_df['facies_merge_1'].values
#groups, categories = pd.factorize(data_df['core_section'])
groups = data_df['core_section'].values

#train_idx, test_idx = my_train_test_split(X, y, groups)
X_train, X_test, y_train, y_test, groups_train, groups_test = my_train_test_split(X, y, groups)

In [21]:
y_test

array([1, 1, 1, ..., 6, 6, 6])

In [5]:
import dask.array as da
X_train = da.from_array(X_train)
y_train = da.from_array(y_train)

In [22]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))

param_grid = {'logisticregression__C': [10**_ for _ in range(-1, 6)]}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

('score-f458167dd7f31df5e9d2386b581470d0', 6, 2) has failed... retrying


KeyError: "('logisticregression-fit-f458167dd7f31df5e9d2386b581470d0', 6, 2)"

The connection to the server lost.

### If not using my cv?

In [6]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV


lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))

param_grid = {'logisticregression__C': [10**_ for _ in range(-4, 6)]}

grid = GridSearchCV(lr, param_grid = param_grid, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

('score-9e5bb3c08b3029278d2b9e31ba6a80e5', 9, 0) has failed... retrying


KeyError: "('pipeline-9e5bb3c08b3029278d2b9e31ba6a80e5', 9, 0)"

The connection to the server lost again.

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

#lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))
lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))
param_grid = {'logisticregression__C': [10**_ for _ in range(-4, 6)]}


mycv = my_group_stratify_shuffle_cv(X, y, groups)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X, y)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

KilledWorker: ("('read-csv-eq-1f89ac024f65b8485983c166e59a7a56', 0)", <Worker 'tcp://10.110.10.26:35217', name: 6-7, memory: 0, processing: 1>)

Okay, still doesn't work. Let me try manual submission.

In [2]:
from split import my_train_test_split
data_df = dd.read_csv('data/XRF_ML_cr.csv')
X = data_df.iloc[:, 1:-2].to_dask_array(lengths=True)
y = data_df['facies_merge_1'].to_dask_array(lengths=True)
groups = data_df['core_section'].to_dask_array(lengths=True)

X_train, X_test, y_train, y_test, groups_train, groups_test = my_train_test_split(X, y, groups)

ValueError: Arrays chunk sizes are unknown: (nan,)

A possible solution: https://docs.dask.org/en/latest/array-chunks.html#unknown-chunks
Summary: to compute chunks sizes, use

   x.compute_chunk_sizes()  # for Dask Array `x`
   ddf.to_dask_array(lengths=True)  # for Dask DataFrame `ddf`

In [5]:
print(X_train.shape)
print(X_test.shape)

(131261, 24)
(22096, 24)


In [8]:
len(groups_test) + len(groups_train) - len(groups)

0

## Logistic regression

In [25]:
%%time
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

lr = LogisticRegression(max_iter = 10000, class_weight = 'balanced', n_jobs=-1)

param_grid = {'C': [10**_ for _ in range(-3, 6)]}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = 45)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 5, 3) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 5, 4) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 1, 4) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 4, 2) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 8, 3) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 2, 2) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 2, 1) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 4, 1) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 4, 3) has failed... retrying
('logisticregression-fit-score-087688c3ee48a5626598dc935da060d4', 8, 4) has failed... retrying
('logisticregression-fit-score-087688c3ee48a562659

KeyboardInterrupt: 

In [76]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))

param_grid = {'logisticregression__C': [10**_ for _ in range(-3, 6)]}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

KeyboardInterrupt: 

In [7]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))

param_grid = {'logisticregression__C': [10**_ for _ in range(-4, 6)]}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

/home/users/aslee/miniconda3/lib/python3.7/site-packages/distributed/worker.py:3382: UserWarning: Large object of size 5.25 MB detected in task graph: 
  (_CVIterableWrapper(cv=[(array([     0,      1, .. ... e, False, True)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)
('score-d4e69ff487e977c5429e39a42cc1e85a', 2, 3) has failed... retrying


KeyError: "('pipeline-d4e69ff487e977c5429e39a42cc1e85a', 2, 3)"

In [6]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_dask_cv

lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))

param_grid = {'logisticregression__C': [10**_ for _ in range(-4, 6)]}

mycv = my_dask_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

ValueError: Arrays chunk sizes are unknown: (nan,)

A possible solution: https://docs.dask.org/en/latest/array-chunks.html#unknown-chunks
Summary: to compute chunks sizes, use

   x.compute_chunk_sizes()  # for Dask Array `x`
   ddf.to_dask_array(lengths=True)  # for Dask DataFrame `ddf`

In [16]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced'))

param_grid = {'logisticregression__C': [10**_ for _ in range(-4, 6)]}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = -1)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

Best score on validation set: 0.58
Best parameters:  {'logisticregression__C': 1}
CPU times: user 47.9 s, sys: 1.87 s, total: 49.8 s
Wall time: 5min 6s


In [17]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

lr = make_pipeline(StandardScaler(), PCA(whiten = True), LogisticRegression(max_iter = 10000, class_weight = 'balanced', n_jobs=-1))

param_grid = {'logisticregression__C': [10**_ for _ in range(-4, 6)]}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(lr, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = 50)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

('score-4957c2b446aa7f4b3938b11fdce7e185', 6, 3) has failed... retrying


KeyError: "('pipeline-4957c2b446aa7f4b3938b11fdce7e185', 6, 3)"

In [30]:
%%time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from dask_ml.model_selection import GridSearchCV
from split import my_group_stratify_shuffle_cv

svc = make_pipeline(StandardScaler(), PCA(whiten = True), SVC(class_weight = 'balanced'))
param_grid = {'svc__C': np.logspace(-3, 5, 9),
             'svc__gamma': np.logspace(-6, 0, 7)}

mycv = my_group_stratify_shuffle_cv(X_train, y_train, groups_train)

grid = GridSearchCV(svc, param_grid = param_grid, cv = mycv, scoring = 'balanced_accuracy', n_jobs = 50)

grid.fit(X_train, y_train)

print("Best score on validation set: {:.2f}".format(grid.best_score_)) 
print("Best parameters: ", grid.best_params_)

('score-87900976ffc67c474bc3f1a52a63a914', 17, 4) has failed... retrying


KeyError: "('svc-fit-87900976ffc67c474bc3f1a52a63a914', 17, 4)"